In [1]:
# Import required libraries
from openai import OpenAI
from dotenv import load_dotenv
import os
import requests
import json

# Load environment variables
load_dotenv()

# Initialize OpenAI client
client = OpenAI(api_key=os.getenv("OPEN_API_KEY"))

def calculate_tax(income, tax_rate):
    tax = income * (tax_rate / 100)
    return {"income": income, "tax_rate": tax_rate, "tax_amount": tax}

def get_weather(location):
    try:
        # First, find the location
        find_url = "https://ai-weather-by-meteosource.p.rapidapi.com/find_places"
        querystring = {"text": location, "language": "en"}
        headers = {
            "x-rapidapi-key": "cd63083a33msha2e5829aa1a8e78p183c42jsn92196e6a83c6",
            "x-rapidapi-host": "ai-weather-by-meteosource.p.rapidapi.com"
        }
        
        response = requests.get(find_url, headers=headers, params=querystring)
        places_data = response.json()
        
        if not places_data:
            return {"error": "Location not found"}
        
        # Get the first (most relevant) location
        place = places_data[0]
        place_id = place['place_id']
        
        # Now get the actual weather data using the place_id
        weather_url = "https://ai-weather-by-meteosource.p.rapidapi.com/current"
        weather_params = {"place_id": place_id, "units": "metric"}
        
        weather_response = requests.get(weather_url, headers=headers, params=weather_params)
        weather_data = weather_response.json()
        
        # Format the response nicely
        return {
            "location": f"{place['name']}, {place['country']}",
            "coordinates": f"{place['lat']}, {place['lon']}",
            "weather_data": weather_data,
            "status": "success"
        }
        
    except Exception as e:
        return {"error": f"Failed to get weather data: {str(e)}"}

# Tool definitions (updated description for weather)
tools = [
    {
        "type": "function",
        "function": {
            "name": "calculate_tax",
            "description": "Calculate tax amount based on income and tax rate",
            "parameters": {
                "type": "object",
                "properties": {
                    "income": {
                        "type": "number",
                        "description": "Annual income amount"
                    },
                    "tax_rate": {
                        "type": "number",
                        "description": "Tax rate as percentage (e.g., 30 for 30%)"
                    }
                },
                "required": ["income", "tax_rate"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get current weather information for a specified location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city name to get weather for"
                    }
                },
                "required": ["location"]
            }
        }
    }
]

# Function calling logic (cleaned up)
def handle_function_calls(messages, tools):
    response = client.chat.completions.create(
        model="gpt-4.1-nano",
        messages=messages,
        tools=tools,
        temperature=0.7
    )
    
    if response.choices[0].message.tool_calls:
        tool_call = response.choices[0].message.tool_calls[0]
        function_name = tool_call.function.name
        function_args = json.loads(tool_call.function.arguments)
        
        # Call appropriate function
        if function_name == "get_weather":
            result = get_weather(function_args["location"])
        elif function_name == "calculate_tax":
            result = calculate_tax(function_args["income"], function_args["tax_rate"])
        else:
            result = {"error": "Unknown function"}
        
        # Update messages with assistant's response and tool result
        messages.append(response.choices[0].message)
        messages.append({
            "role": "tool",
            "tool_call_id": tool_call.id,
            "content": json.dumps(result)  # Ensure proper JSON formatting
        })
        
        # Get final response from the model
        final_response = client.chat.completions.create(
            model="gpt-4.1-nano",
            messages=messages,
            temperature=0.7
        )
        return final_response.choices[0].message.content
    else:
        return response.choices[0].message.content


In [2]:
# Testing both functions
def test_tools():
    # Test tax calculation
    print("=== Tax Calculation Test ===")
    messages = [{"role": "user", "content": "Calculate tax for income 60000 with 30% tax rate"}]
    result = handle_function_calls(messages, tools)
    print(result)
    
    print("\n=== Weather Test ===")
    messages = [{"role": "user", "content": "What's the weather in Delhi?"}]
    result = handle_function_calls(messages, tools)
    print(result)

# Runing tests
test_tools()

=== Tax Calculation Test ===
The tax on an income of $60,000 at a 30% tax rate is $18,000.

=== Weather Test ===
The current weather in Delhi is mostly cloudy with a temperature of around 30°C. The humidity is approximately 38%, and the wind is coming from the southeast at about 3 km/h.
